In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta

def data_processing(data):

#1 to 40
    # Drop irrelavant columns 
    drop_list = ['Unnamed: 0','id','member_id','funded_amnt','url','desc','title']

    drop_for_grade_list = ['funded_amnt_inv','int_rate','installment','issue_d','loan_status','pymnt_plan','out_prncp','out_prncp_inv']

    data = data.drop(drop_list, axis=1)
    data = data.drop(drop_for_grade_list, axis=1)

    # Convert categorical to numerical 
    data['term'] = data['term'].apply(lambda x: int(x.split()[0]))
    data['emp_length'] = data['emp_length'].str.extract('(\d+)') 
    #10 means more than 10 years 

    # Convert to Datetime
    data['earliest_cr_line'] = pd.to_datetime(data['earliest_cr_line'])

    # Missing Values 

    data.mths_since_last_record = data.mths_since_last_record.fillna(0)
    data.mths_since_last_delinq = data.mths_since_last_delinq.fillna(0)

    data.emp_title = data.emp_title.fillna('None')
    data.emp_length = data.emp_length.fillna(0)

    data.revol_util = data.revol_util.fillna(0)

    data.dti = data.dti.fillna(data.revol_bal / data.annual_inc)


#41-80
    data['last_pymnt_d']= pd.to_datetime(data['last_pymnt_d'])
    data['last_credit_pull_d']= pd.to_datetime(data['last_credit_pull_d'])
    data['next_pymnt_d']= pd.to_datetime(data['next_pymnt_d'])
    data.drop(['annual_inc_joint', 'dti_joint', 'verification_status_joint'], axis=1)
    data[['last_pymnt_d', 'next_pymnt_d', 'next_pymnt_d']] = data[['last_pymnt_d', 'next_pymnt_d', 'next_pymnt_d']].replace(np.NaN, 'NA')
    data[['collections_12_mths_ex_med', 'mths_since_last_major_derog']]= data[['collections_12_mths_ex_med', 'mths_since_last_major_derog']].replace(np.NaN, 0)
    data[['acc_now_delinq','tot_coll_amt','open_acc_6m','open_il_6m','open_il_12m']]=data[['acc_now_delinq','tot_coll_amt','open_acc_6m','open_il_6m','open_il_12m']].replace(np.NaN, 0)
    data[['tot_cur_bal']]=data[['tot_cur_bal']].replace(np.NAN, 'NA')
    data[['open_il_24m','open_rv_12m','open_rv_24m','inq_fi']]=data[['open_il_24m','open_rv_12m','open_rv_24m','inq_fi']].replace(np.NaN, 0)
    data['mths_since_rcnt_il'].fillna((data['mths_since_rcnt_il'].mean()), inplace=True)
    data['total_bal_il'].fillna((data['total_bal_il'].mean()), inplace=True)
    data['max_bal_bc'].fillna((data['max_bal_bc'].mean()), inplace=True)
    data['total_rev_hi_lim'].fillna((data['total_rev_hi_lim'].mean()), inplace=True)
    data[['total_cu_tl','inq_last_12m','acc_open_past_24mths','chargeoff_within_12_mths','delinq_amnt']]=data[['total_cu_tl','inq_last_12m','acc_open_past_24mths','chargeoff_within_12_mths','delinq_amnt']].replace(np.NaN, 0)
    data['avg_cur_bal'].fillna((data['avg_cur_bal'].mean()), inplace=True)
    data['bc_open_to_buy'].fillna((data['bc_open_to_buy'].mean()), inplace=True)
    data['bc_util'].fillna((data['bc_util'].mean()), inplace=True)
    data['mo_sin_old_il_acct'].fillna((data['mo_sin_old_il_acct'].mean()), inplace=True)
    data['mo_sin_old_rev_tl_op'].fillna((data['mo_sin_old_rev_tl_op'].mean()), inplace=True)


#81 to 120
    

    #columns to drop
    data.drop(['revol_bal_joint', 'sec_app_fico_range_low','sec_app_fico_range_high','sec_app_earliest_cr_line','sec_app_inq_last_6mths'
                 ,'sec_app_mort_acc'], axis=1, inplace=True)

    #columns to fillna with 0
    fill =data[['bc_util','chargeoff_within_12_mths',
                    'delinq_amnt','mort_acc','mths_since_recent_bc','mths_since_recent_bc_dlq',
                     'num_tl_90g_dpd_24m','num_tl_op_past_12m','pub_rec_bankruptcies','tax_liens']]

    for column in fill:
        data[column].fillna(0)


#120 to rest

       #Drop rows with missing information
    row_to_drop=data[data.issue_d.isna()].index.tolist()
    data=data.drop(data.index[row_to_drop])
    
    #IMPUTE MISSING VALUES
    #for the secondary applicant group, nan is for no secondary applicant. 
    #sec_: numerical columns, -999 is used to distinguish data
    data.sec_app_inq_last_6mths=data.sec_app_inq_last_6mths.fillna(-999)
    data.sec_app_mort_acc=data.sec_app_mort_acc.fillna(-999)
    data.sec_app_open_acc=data.sec_app_open_acc.fillna(-999)
    data.sec_app_revol_util=data.sec_app_revol_util.fillna(-999)
    data.sec_app_open_act_il=data.sec_app_open_act_il.fillna(-999)
    data.sec_app_num_rev_accts=data.sec_app_num_rev_accts.fillna(-999)
    data.sec_app_chargeoff_within_12_mths=data.sec_app_chargeoff_within_12_mths.fillna(-999)
    data.sec_app_collections_12_mths_ex_med=data.sec_app_collections_12_mths_ex_med.fillna(-999)
    data.sec_app_mths_since_last_major_derog=data.sec_app_mths_since_last_major_derog.fillna(-999)
    

    
####!!!NOTE: HARDSHIP AND SETTLEMENTS ARE AFTER LOAN IS ISSUED!!!!
    #Candidate for drop: hardship_flag
    #hardship_flag: Y, N, nan (unique value)
        #observation for N: no hardship (hardship related column is nan) or hardship status is not active (but hardship related columns are filled).
        #observvation for Y: hardship status is active, and all hardship related columns are filled

        
    #fill na but candidate for drop:hardship_type. Only 2 unique category.
    #hardship_type: nan, 'INTEREST ONLY-3 MONTHS DEFERRAL' (unique values)
    data.hardship_type=data.hardship_type.fillna('NA-noHardShip')
    
    #Fill NA when there is no hardship. 
    data.hardship_reason=data.hardship_reason.fillna('NA-noHardShip')
    data.hardship_status=data.hardship_status.fillna('NA-noHardShip')
                              
    #fill na but candidate for drop: Deferall term, only 2 unique category. 
    #deferral_term: nan, 3, (unique values). 
    data.deferral_term=data.deferral_term.fillna(-999)
        
    #Fill -999 when there is no hardship. 
    data.hardship_amount=data.hardship_amount.fillna(-999)
    
    #length_of_hardship_payment:hardship_end_date-payment_plan_start_date
    data.hardship_end_date=data.hardship_end_date.fillna(0)
    data.payment_plan_start_date=data.payment_plan_start_date.fillna(0)
    data.hardship_end_date=pd.to_datetime(data.hardship_end_date)
    data.payment_plan_start_date=pd.to_datetime(data.payment_plan_start_date)
    
    data['length_of_hardship_payment']=(data.hardship_end_date-data.payment_plan_start_date)/np.timedelta64(1, 'M')
    data['length_of_hardship_payment']=data['length_of_hardship_payment'].astype(int)
    data.drop(['hardship_end_date',"payment_plan_start_date"], axis=1)

    #Drop hardship_start_date, hardship_end_date because hardship_length can represent the duration
    data.drop(['hardship_start_date',"hardship_end_date"], axis=1)

    #fill -999 if no hardship
    data.hardship_length=data.hardship_length.fillna(-999)

    #Drop payment_plan_start_date after length_of_hardship_payment imputation
    data.drop(["payment_plan_start_date"],axis=1)
    
    #Fill -999 when there is no hardship.  
    data.hardship_dpd=data.hardship_dpd.fillna(-999)

    #Fill na when no hardship
    data.hardship_loan_status=data.hardship_loan_status.fillna('None')
    
    #by definition, this orig_projected_additional_accrued_interest will be null if hardship_status is 'BROKEN'
    data.orig_projected_additional_accrued_interest=data.orig_projected_additional_accrued_interest.fillna(-999)

    #fill -999 with no hardship
    data.hardship_payoff_balance_amount=data.hardship_payoff_balance_amount.fillna(-999)
    data.hardship_last_payment_amount=data.hardship_last_payment_amount.fillna(-999) 
    
    
####NOTE: SETTLEMENT IS AFTER THE LOAN IS ISSUE
    #Fill na with no settlement involved
    data.settlement_status=data.settlement_status.fillna('None')
    data.settlement_term=data.settlement_term.fillna('None')
    #Fill -999 with no settlement involved
    data.settlement_amount=data.settlement_amount.fillna(-999)
    data.settlement_percentage=data.settlement_amount.fillna(-999)
    
    #Combine debt_settlement_flag_date with settlement_date
    data.debt_settlement_flag_date=data.debt_settlement_flag_date.fillna(0)
    data.settlement_date=data.settlement_date.fillna(0)
    data.debt_settlement_flag_date=pd.to_datetime(data.debt_settlement_flag_date)
    data.settlement_date=pd.to_datetime(data.settlement_date)
    
    data['duration_settlement_debt_se_flag']=(data.debt_settlement_flag_date-data.settlement_date)/np.timedelta64(1, 'M')
    data['duration_settlement_debt_se_flag']=data['duration_settlement_debt_se_flag'].astype(int)
    data.drop(['debt_settlement_flag_date',"settlement_date"], axis=1)

return data
